In [1]:

#---------------------------------------------------------------------
# Importing Libraries
print('---------------------------------------------------------------------------------')
print('Importing Libraries')
print('---------------------------------------------------------------------------------')
print('')

import numpy
import time
import tensorflow as tf
import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math
from sklearn.ensemble import RandomForestClassifier
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from imblearn.over_sampling import RandomOverSampler
import shap
from scipy.special import softmax
np.random.seed(0)
from sklearn.model_selection import train_test_split
import sklearn
#---------------------------------------------------------------------
# Defining metric equations

print('---------------------------------------------------------------------------------')
print('Defining Metric Equations')
print('---------------------------------------------------------------------------------')
print('')
def print_feature_importances_shap_values(shap_values, features):
    '''
    Prints the feature importances based on SHAP values in an ordered way
    shap_values -> The SHAP values calculated from a shap.Explainer object
    features -> The name of the features, on the order presented to the explainer
    '''
    # Calculates the feature importance (mean absolute shap value) for each feature
    importances = []
    for i in range(shap_values.values.shape[1]):
        importances.append(np.mean(np.abs(shap_values.values[:, i])))
    # Calculates the normalized version
    importances_norm = softmax(importances)
    # Organize the importances and columns in a dictionary
    feature_importances = {fea: imp for imp, fea in zip(importances, features)}
    feature_importances_norm = {fea: imp for imp, fea in zip(importances_norm, features)}
    # Sorts the dictionary
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}
    feature_importances_norm= {k: v for k, v in sorted(feature_importances_norm.items(), key=lambda item: item[1], reverse = True)}
    # Prints the feature importances
    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f} (softmax = {feature_importances_norm[k]:.4f})")


def ACC(TP,TN,FP,FN):
    Acc = (TP+TN)/(TP+FP+FN+TN)
    return Acc
def ACC_2 (TP, FN):
    ac = (TP/(TP+FN))
    return ac
def PRECISION(TP,FP):
    Precision = TP/(TP+FP)
    return Precision
def RECALL(TP,FN):
    Recall = TP/(TP+FN)
    return Recall
def F1(Recall, Precision):
    F1 = 2 * Recall * Precision / (Recall + Precision)
    return F1
def BACC(TP,TN,FP,FN):
    BACC =(TP/(TP+FN)+ TN/(TN+FP))*0.5
    return BACC
def MCC(TP,TN,FP,FN):
    MCC = (TN*TP-FN*FP)/(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**.5)
    return MCC
def AUC_ROC(y_test_bin,y_score):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    auc_avg = 0
    counting = 0
    for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
      auc_avg += auc(fpr[i], tpr[i])
      counting = i+1
    return auc_avg/counting

def oversample(X_train, y_train):
    oversample = RandomOverSampler(sampling_strategy='minority')
    # Convert to numpy and oversample
    x_np = X_train.to_numpy()
    y_np = y_train.to_numpy()
    x_np, y_np = oversample.fit_resample(x_np, y_np)

    # Convert back to pandas
    x_over = pd.DataFrame(x_np, columns=X_train.columns)
    y_over = pd.Series(y_np)
    return x_over, y_over


###########################################################################################################################################################################################################################################################################################################################
print('---------------------------------------------------------------------------------')
print('DNN DeepLIFT NSL GLOBAL')
print('---------------------------------------------------------------------------------')
Trial_1 = [
'service_http',
'dst_host_srv_serror_rate',
'dst_host_serror_rate',
'flag_S0',
'serror_rate',
'srv_serror_rate',
'flag_SF',
'same_srv_rate',
'rerror_rate',
'count',
'dst_host_srv_count',
'logged_in',
'dst_host_same_src_port_rate',
'dst_host_rerror_rate',
'dst_host_srv_rerror_rate',
'hot',
'srv_rerror_rate',
'service_private',
'dst_host_count',
'dst_host_same_srv_rate'
]

Trial_2 = [
'logged_in',
'dst_host_rerror_rate',
'Protocol_type_tcp',
'dst_host_srv_rerror_rate',
'srv_rerror_rate',
'dst_host_same_srv_rate',
'dst_host_srv_serror_rate',
'dst_host_serror_rate',
'dst_host_srv_count',
'dst_host_count',
'rerror_rate',
'srv_serror_rate',
'Protocol_type_udp',
'flag_S0',
'flag_SF',
'same_srv_rate',
'service_http',
'flag_REJ',
'count',
'dst_host_same_src_port_rate'
]

Trial_3 = [
'logged_in',
'dst_host_count',
'count',
'service_private',
'service_http',
'srv_diff_host_rate',
'flag_S0',
'srv_serror_rate',
'diff_srv_rate',
'flag_SF',
'rerror_rate',
'flag_REJ',
'same_srv_rate',
'dst_host_rerror_rate',
'dst_host_same_srv_rate',
'srv_count',
'dst_host_srv_rerror_rate',
'service_domain_u',
'dst_host_srv_serror_rate',
'Protocol_type_tcp'
]

Intersection123 = set(Trial_1) & set(Trial_2) & set(Trial_3)

Total = len(Trial_1)

RF_stab = len(Intersection123)/Total
print(RF_stab)

print('---------------------------------------------------------------------------------')
print('DNN IG NSL GLOBAL')
print('---------------------------------------------------------------------------------')


Trial_1 = [
'dst_host_srv_serror_rate',
 'dst_host_serror_rate',
 'service_http',
 'srv_serror_rate',
 'serror_rate',
 'flag_SF',
 'flag_S0',
 'dst_host_srv_count',
 'dst_host_same_src_port_rate',
 'same_srv_rate',
 'dst_host_count',
 'count',
 'logged_in',
 'Protocol_type_icmp',
 'service_private',
 'service_ecr_i',
 'dst_host_rerror_rate',
 'service_smtp',
 'dst_host_srv_rerror_rate',
 'dst_host_diff_srv_rate'
]

Trial_2 = [
'logged_in',
 'dst_host_srv_serror_rate',
 'dst_host_serror_rate',
 'Protocol_type_udp',
 'srv_serror_rate',
 'flag_S0',
 'dst_host_same_srv_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'service_private',
 'flag_SF',
 'serror_rate',
 'dst_host_diff_srv_rate',
 'Protocol_type_tcp',
 'same_srv_rate',
 'dst_host_rerror_rate',
 'count',
 'dst_host_srv_rerror_rate',
 'dst_host_same_src_port_rate',
 'service_http'
]
Trial_3 =[
'dst_host_count',
 'logged_in',
 'srv_serror_rate',
 'flag_S0',
 'count',
 'service_private',
 'srv_diff_host_rate',
 'flag_SF',
 'same_srv_rate',
 'diff_srv_rate',
 'rerror_rate',
 'dst_host_srv_rerror_rate',
 'Protocol_type_tcp',
 'dst_host_same_srv_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_serror_rate',
 'srv_count',
 'service_http',
 'serror_rate',
 'flag_REJ'
]
Intersection123 = set(Trial_1) & set(Trial_2) & set(Trial_3)

Total = len(Trial_1)

RF_stab = len(Intersection123)/Total
print(RF_stab)


print('---------------------------------------------------------------------------------')
print('DNN LRP NSL GLOBAL')
print('---------------------------------------------------------------------------------')


Trial_1 = [
'dst_host_srv_serror_rate',
 'dst_host_serror_rate',
 'service_http',
 'srv_serror_rate',
 'serror_rate',
 'flag_SF',
 'dst_host_srv_count',
 'flag_S0',
 'dst_host_same_src_port_rate',
 'same_srv_rate',
 'dst_host_count',
 'count',
 'logged_in',
 'Protocol_type_icmp',
 'service_private',
 'service_ecr_i',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'service_smtp',
 'dst_host_diff_srv_rate'
]

Trial_2 = [
'dst_host_count',
 'logged_in',
 'srv_serror_rate',
 'count',
 'flag_S0',
 'flag_SF',
 'same_srv_rate',
 'service_private',
 'srv_diff_host_rate',
 'Protocol_type_tcp',
 'dst_host_srv_rerror_rate',
 'rerror_rate',
 'dst_host_same_srv_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_serror_rate',
 'diff_srv_rate',
 'srv_count',
 'service_http',
 'serror_rate',
 'service_domain_u'
]
Trial_3 =[
'logged_in',
 'dst_host_srv_serror_rate',
 'dst_host_serror_rate',
 'srv_serror_rate',
 'Protocol_type_udp',
 'dst_host_same_srv_rate',
 'dst_host_count',
 'flag_S0',
 'dst_host_srv_count',
 'service_private',
 'flag_SF',
 'serror_rate',
 'same_srv_rate',
 'count',
 'dst_host_diff_srv_rate',
 'service_http',
 'dst_host_rerror_rate',
 'dst_host_same_src_port_rate',
 'Protocol_type_tcp',
 'dst_host_srv_rerror_rate'
]
Intersection123 = set(Trial_1) & set(Trial_2) & set(Trial_3)

Total = len(Trial_1)

RF_stab = len(Intersection123)/Total
print(RF_stab)

---------------------------------------------------------------------------------
Importing Libraries
---------------------------------------------------------------------------------



/home/oarreche@ads.iu.edu/anaconda3/envs/HITL/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---------------------------------------------------------------------------------
Defining Metric Equations
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
DNN DeepLIFT NSL GLOBAL
---------------------------------------------------------------------------------
0.65
---------------------------------------------------------------------------------
DNN IG NSL GLOBAL
---------------------------------------------------------------------------------
0.65
---------------------------------------------------------------------------------
DNN LRP NSL GLOBAL
---------------------------------------------------------------------------------
0.65
